## Create 2,000 random latitudes and longitudes

In [3]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy

In [4]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [5]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [6]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

739

## Perform API call with the OpenWeatherMap

In [8]:
# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

# Import the time library and the datetime module from the datetime library 
import time
from datetime import datetime

In [9]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [10]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        for i in city_weather["weather"]:
            desc = i.values()
            desc = list(desc)
            city_description = desc[2]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Current Description": city_description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | presidencia roque saenz pena
Processing Record 2 of Set 1 | rocha
Processing Record 3 of Set 1 | castro
Processing Record 4 of Set 1 | rikitea
Processing Record 5 of Set 1 | vilaka
Processing Record 6 of Set 1 | khasan
Processing Record 7 of Set 1 | east london
Processing Record 8 of Set 1 | sampaloc
Processing Record 9 of Set 1 | kaitangata
Processing Record 10 of Set 1 | hermanus
Processing Record 11 of Set 1 | vaini
Processing Record 12 of Set 1 | hilo
Processing Record 13 of Set 1 | saskylakh
Processing Record 14 of Set 1 | jamestown
Processing Record 15 of Set 1 | barentsburg
City not found. Skipping...
Processing Record 16 of Set 1 | mataura
Processing Record 17 of Set 1 | butaritari
Processing Record 18 of Set 1 | nouadhibou
Processing Record 19 of Set 1 | lugovoy
Processing Record 20 of Set 1 | constitucion
Processing Record 21 of Set 1 | punta arenas
Processing Record 22 of Set 1 | nurot

In [11]:
city_data_df = pd.DataFrame(city_data)

In [13]:
new_columns_order = ["City","Country","Lat","Lng","Max Temp","Humidity","Cloudiness","Wind Speed","Current Description"]
city_data_df = city_data_df[new_columns_order]


In [14]:
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Presidencia Roque Saenz Pena,AR,-26.7852,-60.4388,78.78,64,32,11.92,scattered clouds
1,Rocha,UY,-34.4833,-54.3333,67.60,89,100,5.28,overcast clouds
2,Castro,BR,-24.7911,-50.0119,75.43,60,85,3.74,overcast clouds
3,Rikitea,PF,-23.1203,-134.9692,76.73,74,11,16.35,few clouds
4,Vilaka,LV,57.1846,27.6722,30.78,98,100,3.49,overcast clouds
5,Khasan,RU,42.4308,130.6434,36.48,60,70,3.27,broken clouds
6,East London,ZA,-33.0153,27.9116,69.12,74,26,5.84,scattered clouds
7,Sampaloc,PH,14.5402,121.3602,72.46,93,100,4.65,overcast clouds
8,Kaitangata,NZ,-46.2817,169.8464,58.39,50,100,6.93,overcast clouds
9,Hermanus,ZA,-34.4187,19.2345,63.55,84,75,7.61,broken clouds


In [18]:
# Create the output file (CSV).
output_data_file = "Weather_Database/WeatherPy_Database.csv"

# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file)